In [1]:
import numpy as np
import import_ipynb
from dumbAgent import DumbAgent
from environment import TicTacToe3D
import pygame, sys

importing Jupyter notebook from dumbAgent.ipynb
importing Jupyter notebook from environment.ipynb
pygame 2.5.2 (SDL 2.28.3, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# dummy_board = np.zeros((4, 4, 4), dtype=int)
# # floor, row, col
# dummy_board[0, 0, 0] = 1
# dummy_board[0, 1, 1] = 1
# dummy_board[0, 2, 2] = 1

# agent = DumbAgent()
# x = agent.findBestMove(dummy_board, 1)
# print(x)

In [3]:
agent = DumbAgent()
agent1 = DumbAgent(depth=2)
agent2 = DumbAgent(depth=2)

env = TicTacToe3D(headless=False)
env.draw_lines()

player = 1

MODE = 3
# 1 for player vs player
# 2 for player vs bot
# 3 for bot vs bot

while True:
	if MODE == 1:
		pass
	elif MODE == 2:
		# player vs bot
		command = input()

		if command == 'q':
			pygame.quit()
			break
		else:
			# For Player
			x, y = [int(a) for a in command.split(" ")]
			if not env.move(x, y, player):
				print('Invalid move')
				continue
			print(f"PLAYER: {x}, {y}")
			env.draw_figures()
			player = -player

			if (result := env.check()):
				print('Player', 'OAB'[result], 'wins!')
				break

			# For BOT2
			botX, botyY = agent.findBestMove(env.board, player)
			print(f"BOT: {botX}, {botyY}")
			env.move(botX, botyY, player)
			env.draw_figures()
			player = -player
			
			if (result := env.check()):
				print('Player', 'OAB'[result], 'wins!')
				break
	elif MODE == 3:
		# For BOT1
		mBot = agent1.findBestMove(env.board, player)

		if mBot is None:
			print("No moves left")
			break

		print(f"BOT1: {mBot[0]}, {mBot[1]}")
		env.move(mBot[0], mBot[1], player)
		env.draw_figures()
		player = -player

		if (result := env.check()):
			print('Player', 'OAB'[result], 'wins!')
			break

		# For BOT2
		mBot = agent2.findBestMove(env.board, player)

		if mBot is None:
			print("No moves left")
			break

		print(f"BOT2: {mBot[0]}, {mBot[1]}")
		env.move(mBot[0], mBot[1], player)
		env.draw_figures()
		player = -player

		if (result := env.check()):
			print('Player', 'OAB'[result], 'wins!')
			break



BOT1: 0, 0
BOT2: 0, 0
BOT1: 0, 0
BOT2: 0, 0
BOT1: 1, 1
BOT2: 0, 1
BOT1: 0, 1
BOT2: 0, 1
BOT1: 0, 1
BOT2: 0, 2
BOT1: 0, 2
BOT2: 0, 2
BOT1: 0, 2
BOT2: 0, 3
BOT1: 0, 3
BOT2: 0, 3
BOT1: 0, 3
BOT2: 1, 0
BOT1: 1, 2
BOT2: 1, 0
BOT1: 1, 0
BOT2: 1, 0
BOT1: 1, 3
BOT2: 1, 1
BOT1: 1, 2
BOT2: 1, 1
BOT1: 1, 1
BOT2: 1, 3
BOT1: 1, 2
BOT2: 1, 2
BOT1: 1, 3
BOT2: 1, 3
BOT1: 2, 0
BOT2: 2, 0
BOT1: 2, 0
BOT2: 2, 0
BOT1: 2, 1
BOT2: 2, 1
BOT1: 2, 1
BOT2: 2, 1
BOT1: 2, 2
BOT2: 2, 3
BOT1: 3, 3
Player A wins!
